In [1]:
import os
os.chdir('../')

## Observe variation in the observation space

In [2]:
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
    
config = None
with open('config.json') as f:
    config = json.load(f)


In [3]:
import gym

all_observations = []

In [4]:

env = gym.make('CartPole-v1')

steps = []

for i_episode in range(200):
    observation = env.reset()
    for t in range(1000):
        env.render()
#         print(observation)
        all_observations.append(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            steps.append(t+1)
#             print("Episode finished after {} timesteps".format(t+1))
            break

env.close()


In [5]:
len(all_observations)

4358

In [6]:
import pandas as pd

cols = ['pos', 'velocity', 'angle', 'angular velocity']
df = pd.DataFrame(data=all_observations, columns=cols)

In [7]:
df.describe()

,pos,velocity,angle,angular velocity
count,4358.000000,4358.000000,4358.000000,4358.000000
mean,-0.006138,-0.034693,0.009551,0.064462
std,0.082357,0.518938,0.090671,0.788913
min,-0.446588,-1.614273,-0.209236,-2.802509
25%,-0.044179,-0.384109,-0.044752,-0.396088
50%,-0.005367,-0.022509,0.011472,0.042293
75%,0.036274,0.240318,0.067749,0.590095
max,0.495205,2.155326,0.209062,2.489823


In [8]:
print('theoretical min:', env.observation_space.low)
print('theoretical max:', env.observation_space.high)

theoretical min: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
theoretical max: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


## Analyze game inputs on those observations

In [9]:
from game_interface import GameInterface

game = MockAIGame(env)
gi = GameInterface(game, config)

In [10]:
all_firing_rates = []

for obs in all_observations:
    game.observations = [obs]
    all_firing_rates.append(gi.input_firing_rates()[:4])
    
print(len(all_firing_rates))

4358


In [11]:
df_fr = pd.DataFrame(data=all_firing_rates, columns=['fr_'+c for c in cols])

In [12]:
df_fr.describe()

,fr_pos,fr_velocity,fr_angle,fr_angular velocity
count,4358.000000,4358.000000,4358.000000,4358.000000
mean,24.780772,24.565898,26.191847,25.645865
std,2.941306,6.484911,11.312401,7.884888
min,9.050440,4.821581,0.000000,0.000000
25%,23.422171,20.198642,19.405970,21.039118
50%,24.808320,24.718641,26.433955,25.422935
75%,26.295514,28.003981,33.468654,30.900947
max,42.685878,50.000000,50.000000,49.898230


In [13]:
maxFiring = config['net']['InputMaxRate']
df_interval = pd.DataFrame(data=[[maxFiring/fr for fr in k]for k in all_firing_rates], columns=['int_'+c for c in cols])

/Users/danielhasegan/workspace/netpyne-project/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


In [14]:
df_interval.describe()

,int_pos,int_velocity,int_angle,int_angular velocity
count,4358.000000,4358.000000,4358.000000,4358.000000
mean,2.053138,2.207854,inf,inf
std,0.325521,0.734820,NaN,NaN
min,1.171348,1.000000,1.000000,1.002040
25%,1.901465,1.785460,1.493935,1.618073
50%,2.015453,2.022765,1.891507,1.966728
75%,2.134730,2.475414,2.576527,2.376525
max,5.524593,10.370042,inf,inf


## Analyze steps per episode

In [15]:
df_steps = pd.DataFrame(steps, columns=['steps_per_episode'])
df_steps.describe()

,steps_per_episode
count,200.000000
mean,21.790000
std,11.855745
min,9.000000
25%,14.000000
50%,18.000000
75%,26.000000
max,84.000000


Metric to evalute: rolling median should be higher than 17

In [3]:
import csv
import pandas as pd
items = []
with open('/Users/danielhasegan/workspace/netpyne-project/results/20210617/ActionsPerEpisode.txt') as f:
    for row in csv.reader(f, delimiter='\t'):
        items.append(int(float(row[1])))


In [12]:

STEP = int(len(items) / 4)
for i in range(int(len(items) / STEP)):
    sub_items = items[STEP * i: STEP * (i+1)]
    print(pd.DataFrame(sub_items, columns=['st_per_ep']).describe())

        st_per_ep
count  209.000000
mean    24.148325
std     12.312127
min      9.000000
25%     15.000000
50%     21.000000
75%     29.000000
max     78.000000
        st_per_ep
count  209.000000
mean    22.784689
std      9.901270
min      9.000000
25%     16.000000
50%     21.000000
75%     28.000000
max     67.000000
        st_per_ep
count  209.000000
mean    22.712919
std     11.467088
min      9.000000
25%     14.000000
50%     19.000000
75%     28.000000
max     63.000000
        st_per_ep
count  209.000000
mean    23.909091
std     11.463419
min      9.000000
25%     15.000000
50%     21.000000
75%     30.000000
max     72.000000


In [7]:
len(items) / 4

209.0